# Replaying Old Data

- Work under a separate branch with git (i.e. git checkout -b my-new-branch)

- Make a new folder in Projects (for your specific purposes)

- Copy this ipynb file that folder

- The data is stored on L-Drive

- the main gist of re-playing existing data is:
    - that your first load everything into a big matrix with mne
    - then initialize a specific **amp** (i.e., the "replayamp")
    - then do exactly the same as normal, with a while True loop with calls to amp.get_data()
    

In [1]:
# import all the needed stuff:

import time
import sys
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt

import easygui  # popup windows with buttons made easy
import mne  # EEGLAB for python
from IPython.display import clear_output  # to clear the cell output during while loop
import re  # regular expressions
import pickle  # to save/load data
import dynarray  # a growing numpy array

import logging
logging.basicConfig(level=logging.ERROR)

sys.path.append("../../mushu")  # driver for the amps
sys.path.append("../../mushu/libmushu")
import libmushu

sys.path.append("../../nftools")  # handy stuff needed for NF
from nftools.loopcontrol import LoopState
from nftools.analysis import convert_alld_allm_to_mne
from nftools.analysis import select_part_from_mne_dataset
from nftools.analysis import plot_compare_two_spectra


sys.path.append("../../wyrm")  # real-time data analysis
from wyrm.types import RingBuffer
from wyrm.types import BlockBuffer
from wyrm import io
from wyrm import processing as proc

import scipy
from scipy import signal

from collections import deque  # a FILO list useful for plotting!

In [2]:
# this is the replay notebook - so select a file for playback - this is for eeglab files.

#fn=easygui.fileopenbox(default='*.set')
fn='trial_data_for_mri_cwl_development_withtrend.set'
# fn='eoec_withsins_30hT.set'
#fn='trio2_eoec_outside_before.set'
print(fn)

raw_fromfile = mne.io.read_raw_eeglab(fn)
montage=mne.channels.read_montage('standard_1005', ch_names=raw_fromfile.ch_names)  # always use MNE definitions
raw_fromfile.set_montage(montage)

trial_data_for_mri_cwl_development_withtrend.set
The following EEG sensors did not have a position specified in the selected montage: ['EOG', 'ECG', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6']. Their position has been left untouched.
The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
Events like the following will be dropped entirely: ['boundary', 'Sync On'], 2 in total
42/120 event codes could not be mapped to integers. Use the 'event_id' parameter to map such events manually.
24 events will be dropped because they occur on the same time sample as another event. `mne.io.Raw` objects store events on an event channel, which cannot represent two events on the same sample. You can extract the original event structure using `mne.io.eeglab.read_events_eeglab`. Then, you can e.g. subset the extracted events for constructing epochs.
Data will be preloaded. preload=False or a string preload is not supported when the da

<ipython-input-2-0131e4cfeebe>:9: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EOG', 'ECG', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6']. Their position has been left untouched.
  raw_fromfile = mne.io.read_raw_eeglab(fn)
<ipython-input-2-0131e4cfeebe>:9: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_fromfile = mne.io.read_raw_eeglab(fn)
<ipython-input-2-0131e4cfeebe>:9: RuntimeWarning: Events like the following will be dropped entirely: ['boundary', 'Sync On'], 2 in total
  raw_fromfile = mne.io.read_raw_eeglab(fn)
<ipython-input-2-0131e4cfeebe>:9: RuntimeWarning: 42/120 event codes could not be mapped to integers. Use the 'event_id' parameter to map such events manually.
  raw_fromfile = mne.io.read_raw_eeglab(fn)
<ipython-input-2-0131e4cfeebe>:9: RuntimeWarning: 24 events will be dropped because they occur on the same time sa

The following EEG sensors did not have a position specified in the selected montage: ['EOG', 'ECG', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6']. Their position has been left untouched.


<ipython-input-2-0131e4cfeebe>:11: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EOG', 'ECG', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6']. Their position has been left untouched.
  raw_fromfile.set_montage(montage)


<RawEEGLAB  |  None, n_channels x n_times : 39 x 400001 (80.0 sec), ~119.1 MB, data loaded>

In [3]:
# properties of the recording, and some properties of the While Loop:
fs = raw_fromfile.info['sfreq']
nbchan = raw_fromfile.info['nchan']-1
updateTime = 0.1  # run some kind of calculation every X seconds
buffSize = 1.0  # run calculation on last X seconds of data

In [4]:
# preparation to plot stuff in real time:
%matplotlib qt5 
plt.ion()  # enable widget plots & interactive plots

time_in_plot=2.0  # how much time in x-axis
sy1=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))  # for plotting - the FILO list
sy2=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))  # for plotting - the FILO list
sy3=deque(np.zeros(round(fs/10 * time_in_plot)), round(fs/10 * time_in_plot))  # for plotting - the FILO list

channel_to_plot=1
sx = np.linspace(0, time_in_plot, round(fs * time_in_plot))
sx2 = np.linspace(0, time_in_plot, round(fs/10* time_in_plot))


In [5]:
# real-time data filtering -- high-pass filter (of 1.0 Hz)

f_low = 1.0
# f_high = 15.0
butter_ord = 3
lenchannels = nbchan

#rt_b, rt_a = signal.butter(butter_ord, [f_low / fn, f_high / fn], btype='band')
rt_b_hp, rt_a_hp = signal.butter(butter_ord, 2*f_low/fs, btype='high', analog=False)  # a digital high-pass filter
rt_zi_hp = proc.lfilter_zi(rt_b_hp, rt_a_hp, lenchannels)


In [6]:
# real-time data filtering -- band-pass filter (of 12.0 - 15.0 Hz)

f_low = 12.0
f_high = 15.0
butter_ord = 3
lenchannels = nbchan

#rt_b, rt_a = signal.butter(butter_ord, [f_low / fn, f_high / fn], btype='band')
rt_b_bp, rt_a_bp = signal.butter(butter_ord, [2*f_low/fs, 2*f_high/fs], btype='band', analog=False)  # a digital high-pass filter
rt_zi_bp = proc.lfilter_zi(rt_b_bp, rt_a_bp, lenchannels)

In [7]:
# prepare data for replay (warning: need probably a lot of memory)

mul_factor = 1.0
if 1e-6 in [raw_fromfile.info['chs'][0]['cal'], raw_fromfile.info['chs'][0]['range']]:
    mul_factor = 1.0 / 1e-6

seed_d=raw_fromfile[:-1,:][0] * mul_factor  # scale the data to seed (so no 1e-6 stuff in the replayed data)
seed_d=np.array(seed_d.transpose())
seed_ch=raw_fromfile.ch_names[0:-1]
seed_fs=raw_fromfile.info['sfreq']

# prepare for replay; markers:
seed_mdata=np.transpose(raw_fromfile[-1,:][0])
seed_m=[[i / raw_fromfile.info['sfreq'] * 1000, int(m[0])] for i, m in enumerate(seed_mdata) if m > 0] 

In [8]:
amp = libmushu.get_amp('replayamp')
amp.configure(seed_d, seed_m, seed_ch, seed_fs, realtime=True, blocksize_samples=100)

In [9]:
from rtfilters import HPF, LPF, BPF, MR, CWL, RESAMPLESAFE

In [10]:
hpf=HPF(f=1.0, fs=5000, order=3, nbchan=38)
mr=MR(trsamples=9750, N_thr=5, corr_thr = 0.995, forget=5, highpass=[], ncorrvalues=1000)
lpf=LPF(f=125, fs=5000, order=3, nbchan=38)
resample=RESAMPLESAFE(fs_source=5000, fs_target=500)
#cwl=CWL(seconds_in_window=6.0, tdelay=0.050, ichs=list(range(30)), icws=list(range(32,38)), fs=500, highpass=[3, 1.0, 500], saveglms=False)
cwl=CWL(seconds_in_window=6.0, tdelay=0.050, ichs=[1,2,3,4], icws=[32,33,34], fs=500, highpass=[], saveglms=True)


{'f': 1.0, 'fs': 5000, 'order': 3, 'nbchan': 38}
{'trsamples': 9750, 'N_thr': 5, 'corr_thr': 0.995, 'forget': 5, 'highpass': [], 'ncorrvalues': 1000}
{'f': 125, 'fs': 5000, 'order': 3, 'nbchan': 38}
{'fs_source': 5000, 'fs_target': 500}
{'seconds_in_window': 6.0, 'tdelay': 0.05, 'icws': [32, 33, 34], 'ichs': [1, 2, 3, 4], 'fs': 500, 'highpass': [], 'saveglms': True}


In [11]:
hpf=HPF(f=1.0, fs=5000, order=3, nbchan=38)
#bpf=BPF(f=[12.0, 15.0], fs=5000, order=3, nbchan=38)
#mr=MR(trsamples=10000, N_thr=5, corr_thr = 0.995, forget=6)
mr=MR(trsamples=9750, N_thr=5, corr_thr = 0.995, forget=5, highpass=[], ncorrvalues= 1000)
lpf=LPF(f=125, fs=5000, order=3, nbchan=38)
resample=RESAMPLESAFE(fs_source=5000, fs_target=500)
#cwl=CWL(seconds_in_window=6.0, tdelay=0.050, ichs=list(range(30)), icws=list(range(32,38)), fs=500, highpass=[3, 1.0, 500], saveglms=False)
cwl=CWL(seconds_in_window=6.0, tdelay=0.050, ichs=range(30), icws=range(32,38), fs=500, highpass=[], saveglms=True)

# there are several recommendations to follow when building your MRI artifact removal.
# it IS possible to do DC measurements in the MRI, but signal quality of MRI correction will suffer (only a little bit!) due to necessity of
# applying HPF on data for MRI artifact considerations, and absence of any filter on the data that is corrected.
# better to use hpf separately on data --> THEN MRI correction without any filters, than NO hpf and using hpf within the mr correction
# however, the latter I still implement in case of SCP NF - when you have to use the second option.

{'f': 1.0, 'fs': 5000, 'order': 3, 'nbchan': 38}
{'trsamples': 9750, 'N_thr': 5, 'corr_thr': 0.995, 'forget': 5, 'highpass': [], 'ncorrvalues': 1000}
{'f': 125, 'fs': 5000, 'order': 3, 'nbchan': 38}
{'fs_source': 5000, 'fs_target': 500}
{'seconds_in_window': 6.0, 'tdelay': 0.05, 'icws': range(32, 38), 'ichs': range(0, 30), 'fs': 500, 'highpass': [], 'saveglms': True}
saving GLM: glm1552494947
saving GLM: glm1552494950
saving GLM: glm1552494953
saving GLM: glm1552494957
saving GLM: glm1552494959
saving GLM: glm1552494962
saving GLM: glm1552494965
saving GLM: glm1552494968
saving GLM: glm1552494971
saving GLM: glm1552494975
saving GLM: glm1552494977
saving GLM: glm1552494980
saving GLM: glm1552494984
saving GLM: glm1552494986
saving GLM: glm1552494989
saving GLM: glm1552494992
saving GLM: glm1552494995
saving GLM: glm1552494999
saving GLM: glm1552495001
saving GLM: glm1552495004
saving GLM: glm1552495007
saving GLM: glm1552495010
saving GLM: glm1552495013
saving GLM: glm1552495016
saving

In [12]:
fname='blabla'

In [13]:
'saving: %s' % fname

'saving: blabla'

In [14]:
amp.get_sampling_frequency()

5000.0

In [15]:
amp.start()

In [16]:
def runLoop():
    alld=dynarray.DynamicArray((None, len(amp.get_channels())))     # the growing numpy data matrix
    allm=[]     # markers
    sfreq = amp.get_sampling_frequency()  # sampling frequency
    ch_names=amp.get_channels()  # channel names

    rb = RingBuffer(buffSize * 1000)  # the buffer containing the last X seconds of data - declared in MILISECONDS
    totalTime = seed_d.shape[0]/raw_fromfile.info['sfreq']

    fig=plt.figure()  # plotting...
    th=fig.suptitle('')
    ah1=fig.add_subplot(131)
    ah2=fig.add_subplot(132)
    ah3=fig.add_subplot(133)
    l1, = ah1.plot(sx, sy1)
    l2, = ah2.plot(sx, sy2)
    l3, = ah3.plot(sx2, sy3)


    # l=LoopState(); l.start()
    markeroffset = 0  # needed to store all data in one big mat/vector
    t0=time.time()
    curTime=time.time()
    markTime=time.time()
    st=''
    i=0; fnames=[] # for making a movie...
    while curTime - t0 < totalTime:  # l.get_state() != 'Stop':


        # keep track of time:
        curTime = time.time()

        # this is where you get the data
        data, marker = amp.get_data()


        if data.shape[0] > 0:  # this is crucial for remembering filter state.


            data2=hpf.handle(data)
            data3=mr.handle(data2)
            data4=lpf.handle(data3)
            data5=resample.handle(data4)
            data6=cwl.handle(data5)

            # something like this:
            #filterchain = [HPFilter, MRFilter, LPFilter, ResampleFilter, HPFilter, CWLFilter]

            #corr_data = ProcessFilters(chain, (data, marker))


            # use case -- first using the MR Corrector
            #mr_data = MRFilter.filter(data)

            # then -- using the CWL corrector
            #cwl_mr_data = CWLFilter.filter(data)


            #dataf, rt_zi_bp = signal.lfilter(rt_b_bp, rt_a_bp, data, axis=0, zi=rt_zi_bp)  # how to operate directly on the data

            cnt = io.convert_mushu_data(data, marker, sfreq, ch_names)
            mr_cnt = io.convert_mushu_data(data3, marker, sfreq, ch_names)
            cwl_cnt = io.convert_mushu_data(data6, marker, sfreq, ch_names)

            # f_cnt, rt_zi_bp = proc.lfilter(cnt, rt_b_bp, rt_a_bp, zi=rt_zi_bp)  # real-time data preprocessing...

            # plotting...
            sy1.extend(cnt.data[:,channel_to_plot])  # to visualize/plot -- s1 and s2 are deque's
            sy2.extend(mr_cnt.data[:,channel_to_plot])
            sy3.extend(cwl_cnt.data[:,channel_to_plot])


            l1.set_ydata(sy1)
            l2.set_ydata(sy2)
            l3.set_ydata(sy3)
            msy1=np.mean(sy1)
            msy2=np.mean(sy2)
            msy3=np.mean(sy3)
            ah1.set_ylim(-5000+msy1, 5000+msy1)
            ah2.set_ylim(-250+msy2, 250+msy2)
            ah3.set_ylim(-250+msy3, 250+msy3)

            fig.canvas.draw()
            fig.canvas.flush_events()

            # i+=1; fname = '_tmp%03d.png' % i; plt.savefig(fname); fnames.append(fname)

            # currently has no purpose
            newsamples = cnt.data.shape[0]

            # append to ringbuffer, so we can calculate features later on on the last N secs/samples of data.
            rb.append(cwl_cnt)

            # append it to the big matrix, for saving later on with pickle.
            alld.extend(data6)
            for m in marker:
                allm.append([m[0] + markeroffset, m[1]])
            markeroffset += newsamples / float(sfreq) * 1000.



            # do the following every 0.1 msec - with with the ringbuffer:
            if curTime - markTime > updateTime:
                # do Stuff

                markTime = curTime
                # 1) obtain last 1-second(s)
                d = rb.get()



                # clear_output(wait=True)  # write some logging information here
                # clear_output clear the output of the cell, but if you do that you also remove the figures, it seems
                # so don't do it!
                str1 = 'Playing Back - time = %f' % (curTime - t0)
                str2 = 'Length Markers: %d' % len(allm)
                str3 = '%d, %d' % data.shape
                #str4 = 'Feature Value: %f' % feature
                #str5 = 'Scaled Signal for NF: %f' % signalToSend
                #print(str1 + '\n' + str2 + '\n' + str3 + '\n' + str4 + '\n' + str5)

                # print('Length Markers: %d' % len(allm))
                # print(data.shape)
                th.set_text(str1 + '\n' + str2 + '\n' +str3)
                #featureth.set_text(str4 + '\n' + str5)

In [17]:
runLoop()

In [18]:
%load_ext line_profiler

In [19]:
%lprun -s -m rtfilters runLoop()

Timer unit: 1e-06 s

Total time: 0.032061 s
File: /home/johan/.conda/envs/rt/lib/python3.6/site-packages/scipy/stats/stats.py
Function: pearsonr at line 2940

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2940                                           def pearsonr(x, y):
  2941                                               r"""
  2942                                               Calculate a Pearson correlation coefficient and the p-value for testing
  2943                                               non-correlation.
  2944                                           
  2945                                               The Pearson correlation coefficient measures the linear relationship
  2946                                               between two datasets. Strictly speaking, Pearson's correlation requires
  2947                                               that each dataset be normally distributed, and not necessarily zero-mean.
  2948                          

In [20]:
amp.stop()

amplifier stopped!


In [16]:
np.array(cwl._beta_shared_arr)

array([-0.05161199,  0.16482881, -0.0890512 , ..., -0.00238527,
       -0.02011766,  0.3785572 ])

In [20]:
import multiprocessing
Q=multiprocessing.Queue()

In [21]:
Q.get_nowait?

Signature: Q.get_nowait()
Docstring: <no docstring>
File:      ~/.conda/envs/rt/lib/python3.6/multiprocessing/queues.py
Type:      method


In [ ]:
import ipdb
ipdb.post_mortem(sys.last_traceback)

> /home/johan/nf/nf-rtime/Projects_Templates/3_RTMRICWL/rtfilters.py(1038)handle()
   1037         else:
-> 1038             self.buff=np.concatenate(self.buff, data)
   1039 



ipdb>  self.buff


array([], shape=(0, 38), dtype=float64)


ipdb>  data


array([[ 659.41060199,  658.6991346 ,  659.25327234, ...,  647.66778474,
         631.96821966,  632.31727003],
       [ 659.04145416,  658.08179843,  658.75489601, ...,  647.60889236,
         631.64102415,  631.84603206],
       [ 658.57498932,  657.59001058,  658.21078241, ...,  647.4740846 ,
         631.42746428,  631.37870809],
       ...,
       [-197.38515447, -181.40978434, -188.47218639, ..., -181.74570221,
        -175.81218772, -173.18305106],
       [-197.74442442, -181.68199867, -188.17221261, ..., -181.88669696,
        -175.62863734, -173.44709353],
       [-198.13175727, -182.12629108, -187.84575362, ..., -182.01466308,
        -175.43989604, -173.81104621]])


...

38


ipdb>  np


<module 'numpy' from '/home/johan/.conda/envs/rt/lib/python3.6/site-packages/numpy/__init__.py'>


ipdb>  np.concatenate?


*** SyntaxError: invalid syntax


In [14]:
import subprocess
print('Making movie animation.mpg - this may take a while')
subprocess.call("mencoder 'mf://_tmp*.png' -mf type=png:fps=10 -ovc lavc "
                "-lavcopts vcodec=wmv2 -oac copy -o animation.mpg", shell=True)

Making movie animation.mpg - this may take a while


0

In [15]:
%load_ext line_profiler

In [16]:
def bla(var1, var2):
    print(var1)

In [22]:
class bla2():
    def __init__(self, var1):
        self.var1=var1
        
    def print(self):
        print(self.var1)

In [75]:
%%time
print(1)
print(2)

1
2
CPU times: user 341 µs, sys: 0 ns, total: 341 µs
Wall time: 344 µs


In [25]:
b=bla2('1')
b.print()

1


In [50]:
t = %lprun -r -f b.print b.print()

1


Timer unit: 1e-06 s

Total time: 0.001226 s
File: <ipython-input-22-c5111973fb79>
Function: print at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                               def print(self):
     6         1       1226.0   1226.0    100.0          print(self.var1)

In [51]:
t

In [15]:
data5.shape

(280, 2)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
np.matrix

In [2]:
sys.executable

'/home/johan/.conda/envs/rt/bin/python'

In [3]:
import numpy as np

In [ ]:
np.ndarray

In [6]:
m=np.random.random((3,3))

In [ ]:
np.random.random((3,3)).copy

In [20]:
plt.figure()

<Figure size 640x480 with 0 Axes>

In [22]:
plt.plot(cwl.betas[-1][::4,:])

In [15]:
import ipdb
ipdb.post_mortem(sys.last_traceback)

> /home/johan/nf/nf-rtime/Projects_Templates/3_RTMRICWL/rtfilters.py(758)_check_switch_betas()
    757         if self._queue_incoming_betas.qsize() > 0:
--> 758             betas, diff1, diff2 = self._queue_incoming_betas.qsize.get()
    759 



ipdb>  self._queue_incoming_betas


ipdb>  self._queue_incoming_betas.qsize()


1


ipdb>  a, b, c = self._queue_incoming_betas.get()


self = <rtfilters.CWL object at 0x7f6da80d8630>


ipdb>  a


self = <rtfilters.CWL object at 0x7f6da80d8630>


ipdb>  b
ipdb>  c


In [53]:
1+

SyntaxError: invalid syntax (<ipython-input-53-c913c1dffb59>, line 1)

In [57]:
%lprun -f bla bla(1,2)

1


Timer unit: 1e-06 s

Total time: 0.000307 s
File: <ipython-input-16-1dd24238f230>
Function: bla at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def bla(var1, var2):
     2         1        307.0    307.0    100.0      print(var1)

In [58]:
def test():
    %rerun 57


In [71]:
def f(): 
    %rerun 57
%lprun -f bla f()

=== Executing: ===
%lprun -f bla bla(1,2)
=== Output: ===
1


Timer unit: 1e-06 s

Total time: 0 s
File: <ipython-input-16-1dd24238f230>
Function: bla at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def bla(var1, var2):
     2                                               print(var1)

In [16]:
import traceback as tb

In [16]:
import pdb; pdb.post_mortem(tb)

NameError: name 'tb' is not defined

In [15]:
lu=[15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [ ]:
sum([4 > i for i in [0, 1, 2, 3, 4, 5]])

In [ ]:
lu[1:1+3]?

In [19]:
mark=[i for i, lastused in enumerate(lu) if lastused > 10]

In [16]:
help

Type help() for interactive help, or help(object) for help about object.

In [16]:
for popi in reversed(mark):
    lu.pop(popi)

In [17]:
lu

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [4]:
len(l1)

39

In [2]:
l1=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [3]:
l2=[-0.009725803730136564, -0.013286856216523535, -0.0015234020565639168, 0.007469876784025759, 0.21877457609218673, 0.060098715105399894, -0.01152044543151802, -0.010705063562098325, 0.06596197714099682, 0.2209395760192517, 0.03471496614715212, -0.010944518706039706, 0.1077332572705931, 0.9385757350172169, 0.10389958737708403, -0.010958975008754477, 0.034747485560958864, 0.2137010394744165, 0.058628170832456, -0.00995803061856474, -0.009521755934525783, 0.06403311973637298, 0.21299163216259706, 0.0334891349327988, -0.010564141057113813, 0.10636356482254435, 0.9339726306369603, 0.10463196522428422, -0.01063869291601751, 0.03586258392270768, 0.22050137141841994, 0.05935932319157627, -0.010372115253860061, -0.008732075113004207, 0.06545691207213597, 0.2204034773349035, 0.03627234910415857, -0.010316740351968527, 0.11455022632253853]

In [19]:
plt.figure()
plt.plot(l2)
plt.show()

In [28]:
print([1,2,3][::-1])

[3, 2, 1]


In [36]:
list(reversed([]))

[]

In [ ]:
[].reverse

In [31]:
[][::-1]

[]

In [19]:
mark=[]
print(mark)
print(mark.reverse())
for popi in mark: #.reverse():
    print(1)

[]
None


In [17]:
data4

array([[-25. , -29. ,   4.5, ..., 104.5,  63. ,  51.5],
       [-24.5, -30. ,   4. , ..., 104.5,  62. ,  50.5],
       [-24.5, -29.5,   4. , ..., 104. ,  62. ,  51. ],
       ...,
       [  1.5,  44. , -29. , ..., -35. ,  48.5,  52.5],
       [  2.5,  43.5, -28.5, ..., -34.5,  49. ,  52.5],
       [  3. ,  42.5, -27.5, ..., -34. ,  48.5,  52. ]])

In [18]:
mark=[x for x in []]

In [20]:
for i in mark:
    print(i)

In [14]:
10000-9900 + 300

400

In [13]:
amp.stop()
alld.shrink_to_fit()

amplifier stopped!


In [1]:
a, b, c = []

ValueError: not enough values to unpack (expected 3, got 0)

In [22]:
# write to disk, so we can re-load it later:

t={'alld':alld, 'allm':allm, 'ch_names':ch_names, 'sfreq':sfreq}
with open('c-allm-and-alld.pkl', 'wb') as f:
    pickle.dump(t, f)

In [16]:
# load from disk:

with open('c-allm-and-alld.pkl','rb') as f:
    t=pickle.load(f)
for key in t.keys():
    locals()[key] = t[key]

In [23]:
raw = convert_alld_allm_to_mne(alld, allm, ch_names, sfreq)  # covert to MNE
# raw.resample(1000)

The following EEG sensors did not have a position specified in the selected montage: ['EOG', 'ECG', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6']. Their position has been left untouched.


../../nftools/nftools/analysis.py:31: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EOG', 'ECG', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6']. Their position has been left untouched.
  montage=montage


Creating RawArray with float64 data, n_channels=38, n_times=400001
    Range : 0 ... 400000 =      0.000 ...    80.000 secs
Ready.
5000.0
Creating RawArray with float64 data, n_channels=1, n_times=400001
    Range : 0 ... 400000 =      0.000 ...    80.000 secs
Ready.


In [24]:
raw.plot(scalings='auto');

In [19]:
# raw.set_eeg_reference(ref_channels='average')
# better not (yet) - before removing bad channels, since these mess up your data big time: see PREP paper:


In [16]:
picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False,
                       stim=False, exclude='bads')

raw.notch_filter(np.arange(50, 300, 50), picks=picks, filter_length='auto', phase='zero')
# add it (potentialy) some other preprocessing steps here

Setting up band-stop filter
Filter length of 33001 samples (6.600 sec) selected


<RawArray  |  None, n_channels x n_times : 39 x 400001 (80.0 sec), ~119.1 MB, data loaded>

In [17]:
raw.plot_psd(tmax=np.inf, fmax=1000, n_fft=2048*4);

Effective window size : 1.638 (s)


/home/johan/.conda/envs/rt/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
/home/johan/.conda/envs/rt/lib/python3.6/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)
/home/johan/.conda/envs/rt/lib/python3.6/site-packages/mne/viz/evoked.py:162: RuntimeWarning: invalid value encountered in maximum
  rgb /= np.maximum(rgb.max(0), 1e-16)  # avoid div by zero
/home/johan/.conda/envs/rt/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/johan/.conda/envs/rt/lib/python3.6/site-packages/matplotlib/colors.py:251: RuntimeWarning: invalid value encountered in less
  if np.any((result < 0) | (result > 1)):
/home/johan/.conda/envs/rt/lib/python3.6/site-packag

In [29]:
mne.viz.plot_sensors(raw.info, show_names=True, ch_type='eeg');

/home/johan/.conda/envs/rt/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [27]:
raw.info['chs']

[{'loc': array([-0.0502438,  0.0531112,  0.042192 ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ]),
  'unit_mul': 0,
  'range': 1.0,
  'cal': 1.0,
  'kind': 2,
  'coil_type': 1,
  'unit': 107,
  'coord_frame': 0,
  'ch_name': 'F3',
  'scanno': 1,
  'logno': 1},
 {'loc': array([0.0518362, 0.0543048, 0.040814 , 0.       , 0.       , 0.       ,
         0.       , 0.       , 0.       , 0.       , 0.       , 0.       ]),
  'unit_mul': 0,
  'range': 1.0,
  'cal': 1.0,
  'kind': 2,
  'coil_type': 1,
  'unit': 107,
  'coord_frame': 0,
  'ch_name': 'F4',
  'scanno': 2,
  'logno': 2},
 {'loc': array([-0.0653581, -0.0116317,  0.064358 ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ]),
  'unit_mul': 0,
  'range': 1.0,
  'cal': 1.0,
  'kind': 2,
  'coil_type': 1,
  'unit': 107,
  'coord_frame': 0,
  'ch_name': 'C3',
  'scanno': 3,
  'logno':

In [ ]:
mne.viz.plot_sensors?